In [1]:
import requests
import os

from typing import Optional, Tuple

url, auth = 'http://131.155.21.174:8082', ('', 'ilovecrypto')
sid, token = 20221886, 'de9042e7f99da5cd28917b58a705843082d9354a'

req = requests.Session()
req.cookies['token'] = token

def encrypt(nonce: bytes, plain: bytes) -> Optional[Tuple[bytes, bytes]]:
    assert hasattr(nonce,'hex') and hasattr(plain,'hex') and len(nonce) == 16
    r = req.post(url+'/encrypt', auth=auth, data={'nonce': nonce.hex(), 'plain': plain.hex()})
    if r.text.startswith('Error:'):
        print(r.text)
        return None
    ciphertext, tag = map(bytes.fromhex, r.text.strip().split())
    return ciphertext, tag


def decrypt(nonce: bytes, cipher: bytes, tag: bytes) -> Optional[bytes]:
    assert hasattr(nonce,'hex') and hasattr(cipher,'hex') and hasattr(tag,'hex') and len(nonce) == len(tag) == 16
    r = req.post(url+'/decrypt', auth=auth, data={'nonce': nonce.hex(), 'cipher': cipher.hex(), 'tag': tag.hex()})
    if r.text.startswith('Error:'):
        print(r.text)
        return None
    if 'INVALID' in r.text:
        return None
    return bytes.fromhex(r.text.strip())

def forgery(nonce: bytes, cipher: bytes, tag: bytes) -> None:  # use this to submit your forgery once you've created it
    assert type(nonce) == type(cipher) == type(tag) == bytes
    assert len(cipher) >= 33 and len(nonce) == len(tag) == 16
    r = req.post(url+'/validate_forgery', auth=auth, data={'nonce': nonce.hex(), 'cipher': cipher.hex(), 'tag': tag.hex()})
    if r.text.startswith('Error:'):
        print(r.text)
    print('\x1b[32m{}\x1b[0m'.format(r.text.strip()))

def random_nonce() -> bytes:
    return os.urandom(16)

# convert a number in [0..2^128) to a 16-byte block
def n2b(n: int) -> bytes:
    assert 0 <= n < 2**128
    return int.to_bytes(n, 16, 'big')

# convert a 16-byte block to a number in [0..2^128)
def b2n(b: bytes) -> int:
    assert len(b) == 16
    return int.from_bytes(b, 'big')

# XOR bytes
def xor(xs:bytes, ys: bytes) -> bytes:
    assert len(xs) == len(ys)
    return bytes(x^y for x,y in zip(xs,ys))

In [2]:
# convert a number in [0..2^128) to a 16-byte block
def n2b_64(n: int) -> bytes:
    # assert 0 <= n < 2**128
    return int.to_bytes(n, 64, 'big')

# convert a 16-byte block to a number in [0..2^128)
def b2n_64(b: bytes) -> int:
    # assert len(b) == 64
    return int.from_bytes(b, 'big')

In [3]:
################################################################

# let's encrypt a plaintext and try decrypting a correct and invalid ciphertext.

nonce = random_nonce()
print('nonce: {} ({})'.format(nonce.hex(), repr(nonce)))

plain = b"Hi! Just trying this."
print('plaintext: {} ({})'.format(plain.hex(), repr(plain)))


cipher, tag = encrypt(nonce, plain)
print('ciphertext: {} ({})'.format(cipher.hex(), repr(cipher)))
print('tag: {} ({})'.format(tag.hex(), repr(tag)))

plain2 = decrypt(nonce, cipher, tag)
assert plain2 is not None
print('decrypted (correct): {} ({})'.format(plain2.hex(), repr(plain2)))

# let's flip a bit
cipher = bytes([cipher[0] ^ 1]) + cipher[1:]
plain3 = decrypt(nonce, cipher, tag)
print('decrypted (invalid): {}'.format(plain3))

nonce: 29d0fb5194bb998dd2cd53e5ae439eef (b')\xd0\xfbQ\x94\xbb\x99\x8d\xd2\xcdS\xe5\xaeC\x9e\xef')
plaintext: 486921204a75737420747279696e6720746869732e (b'Hi! Just trying this.')
ciphertext: cb7d75e59a3ca82a1c2f9986c9a15eae187da4f482 (b'\xcb}u\xe5\x9a<\xa8*\x1c/\x99\x86\xc9\xa1^\xae\x18}\xa4\xf4\x82')
tag: 41c0a9ab14496f7a8e8dd5159e43fdf2 (b'A\xc0\xa9\xab\x14Ioz\x8e\x8d\xd5\x15\x9eC\xfd\xf2')
decrypted (correct): 486921204a75737420747279696e6720746869732e (b'Hi! Just trying this.')
decrypted (invalid): None


In [4]:
###########################

In [156]:
nonce_bytes = random_nonce()
nonce_hex = nonce_bytes.hex()
nonce_bytes
nonce_number = b2n(nonce_bytes)
nonce_number

99658243781922747929894542049768960459

In [157]:
message_bytes = os.urandom(64)
message_hex = message_bytes.hex()
message_bytes

b'\x8d6\xd6\x8e,$\x92?\xef\xfd\x19\x92\xb9]}\x8c\xc5\xabV\x82\xce\x96\xbd\x86~y\x10U\xef\xa6\xeb%\xdb1FF\xed-d\xf6\xa1\x9cI\x17g\xc3\xd5\xd3\xcd\xd4[L:|\x86A\x06\xac\xda\xc6\xd12\x02\x92'

In [158]:
message_number = b2n_64(message_bytes)
message_bytes == n2b_64(message_number)
message_array = []
for i in range(0, 64, 16):
    block = message_bytes[i:i+16]
    message_array.append(block)
message_array

[b'\x8d6\xd6\x8e,$\x92?\xef\xfd\x19\x92\xb9]}\x8c',
 b'\xc5\xabV\x82\xce\x96\xbd\x86~y\x10U\xef\xa6\xeb%',
 b'\xdb1FF\xed-d\xf6\xa1\x9cI\x17g\xc3\xd5\xd3',
 b'\xcd\xd4[L:|\x86A\x06\xac\xda\xc6\xd12\x02\x92']

In [159]:
# print(message_array[-2])
# print(b"".join(message_array))
# M[m-1]
message_array[-2] = n2b(int(128))
# M[m]
# If it does not work, try with 16
message_array[-1] = n2b(int(2**128 - 1))
message_array

[b'\x8d6\xd6\x8e,$\x92?\xef\xfd\x19\x92\xb9]}\x8c',
 b'\xc5\xabV\x82\xce\x96\xbd\x86~y\x10U\xef\xa6\xeb%',
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80',
 b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff']

In [160]:
b2n_64(message_bytes)

7395988370414512766327166272923899146894457685277051283517072879441430520033461152565527457204053500720068839007156641305925558478395053374651367771210386

In [161]:
# Remake message in bytes
message_back = b"".join(message_array)
b2n_64(message_back)

7395988370414512766327166272923899146894457685277051283517072879441430520033362008991147129494833931194038974865438883969222475774285745484877207351328767

In [162]:
cyphertext, tag = encrypt(nonce_bytes, message_back)
# plaintext = decrypt(nonce_bytes, cyphertext, tag)
# plaintext == message_back
# list(cyphertext)

In [163]:
# M = M[1]    || M[m-2]   || M[m-1]   || M[m]
# M = M[0:16] || M[16:32] || M[32:48] || M[48:64]
cyphertext_forge_array = []
for i in range(0, 32, 16):
    block = cyphertext[i:i+16]
    cyphertext_forge_array.append(block)
    print(f"len {len(block)}")
cyphertext_forge_array

len 16
len 16


[b'\xaa\xbbMw%k\xa0,|\xe4\x9fUT\xf5\xc8\xfa',
 b'\xea\x99\xd5\xc1{\xec\xc3}\x92\x02/\xd2\x8fpXG']

In [164]:
cyphertext

b'\xaa\xbbMw%k\xa0,|\xe4\x9fUT\xf5\xc8\xfa\xea\x99\xd5\xc1{\xec\xc3}\x92\x02/\xd2\x8fpXG$\x80\xd6P\xcb\x91\xfeDEl\x9a\xe5\xa5{-.\xa0\x0e\xdf\x0c?D\xeal\xdev\xa1%\x9e\xff\xaf\xc0'

In [165]:
def compute_m_1(cyphertext, message):
    # left = C[m-1]
    left = cyphertext[32:48]
    # left = cyphertext[48:64]
    # middle = n2b(b2n(message_back[48:64]))
    middle = n2b(int(128))
    # right_sum = n2b(b2n(cyphertext[0:16]) + b2n(cyphertext[16:32]))
    right_sum = xor(message_back[0:16], message_back[16:32])
                    # + b2n(cyphertext[32:48]))
    # print(f"len left {len(left)} middle {len(middle)} right {len(right_sum)}")
    # print(f"type left {type(left)} middle {type(middle)} right {type(right_sum)}")
    # print(f"left {left}")
    # print(f"middle {middle}")
    first = xor(left, middle) 
    return xor(first, right_sum)
# add C'[m-1]
cyphertext_forge_array.append(compute_m_1(cyphertext, message_back))
# add C'[m]
# cyphertext_forge_array.append(cyphertext[48:64])
forge_text = b"".join(cyphertext_forge_array)

In [166]:
T_prime = xor(message_back[48:64], cyphertext[48:64])
T_prime
assert message_array[3] == message_back[48:64]

In [167]:
print(f"len nonce_bytes {len(nonce_bytes)} forge_text {len(forge_text)} T_prime {len(T_prime)}")
print(f"type nonce_bytes {type(nonce_bytes)} forge_text {type(forge_text)} T_prime {type(T_prime)}")
assert len(forge_text) == 48

len nonce_bytes 16 forge_text 48 T_prime 16
type nonce_bytes <class 'bytes'> forge_text <class 'bytes'> T_prime <class 'bytes'>


In [168]:
# def forgery(nonce: bytes, cipher: bytes, tag: bytes) -> None:  # use this to submit your forgery once you've created it
forgery(nonce_bytes, forge_text, T_prime)

Well done! We are accepting this authenticated ciphertext as a forgery. The tale of your success has been saved to disk.
